# 1-2 Variational AutoEncoder

<img src="./img/vae.png" alt="variationalautoencoder" width="500" align="left"/>

In [ ]:
import os
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import gridspec as gridspec
from tensorflow.contrib.tensorboard.plugins import projector

In [ ]:
CKPT_DIR = "../generated_output/VAEPROJ"

In [ ]:
LEARNING_RATE = 1e-4
TRAINING_STEPS = 3000
BATCH_SIZE = 100
TRAINING_SAMPLES = TRAINING_STEPS * BATCH_SIZE
TRAINING_EPOCHS = TRAINING_SAMPLES / 60000

In [ ]:
IMAGE_DIM = 784
LATENT_DIM = 10
ENCODER_HIDDEN_DIM = [256]
DECODER_HIDDEN_DIM = [256]
graph = tf.Graph()

In [ ]:
def progress_bar(current, total, prefix='', suffix='', decimals=1, length=50, bar=u"\u25AF", fill=u"\u25AE"):
    percent = ("{0:." + str(decimals) + "f}").format(100 * (current / float(total)))
    filledLength = int(length * current // total)
    bar = fill * filledLength + bar * (length - filledLength)
    print('\r%s [%s] %s%% %s' % (prefix, bar, percent, suffix), end = '\r')
    if current == total: 
        print()

[batch_size, 784]

$\rightarrow$ Dense(784, 256) $\rightarrow$ relu $\rightarrow$ [batch_size, 256]

$\rightarrow$ 2 x Dense(256, 10) $\rightarrow$ 2 x [batch_size, 10]

In [ ]:
def encoder_model(features):
    with tf.variable_scope('encoder', reuse=tf.AUTO_REUSE):
        net = features
        for units in ENCODER_HIDDEN_DIM:
            net = tf.layers.dense(
                net, units, 
                activation=tf.nn.relu, 
                kernel_initializer=tf.initializers.he_normal())
        latent_mean = tf.layers.dense(
            net, LATENT_DIM, 
            kernel_initializer=tf.initializers.he_normal())
        latent_log_var = tf.layers.dense(
            net, LATENT_DIM, 
            kernel_initializer=tf.initializers.he_normal())
        return latent_mean, latent_log_var

2 x [batch_size, 10] $\rightarrow$ [batch_size, 10]

In [ ]:
def sampler_model(latent_mean, latent_log_var):
    with tf.variable_scope('sampler', reuse=tf.AUTO_REUSE):
        snd_sample = tf.random_normal(
            tf.shape(latent_log_var), dtype=tf.float32, 
            mean=0., stddev=1.0)
        latent_std = tf.exp(latent_log_var / 2)
        latent = latent_mean + latent_std * snd_sample
        return latent

[batch_size, 10]

$\rightarrow$ Dense(128, 256) $\rightarrow$ relu $\rightarrow$ [batch_size, 256]

$\rightarrow$ Dense(256, 784) $\rightarrow$ sigmoid $\rightarrow$ [batch_size, 784]

In [ ]:
def decoder_model(features):
    with tf.variable_scope('decoder', reuse=tf.AUTO_REUSE):
        net = features
        for units in DECODER_HIDDEN_DIM:
            net = tf.layers.dense(
                net, units, 
                activation=tf.nn.relu, 
                kernel_initializer=tf.initializers.he_normal())
        recon = tf.layers.dense(
            net, IMAGE_DIM, 
            activation=tf.nn.sigmoid, 
            kernel_initializer=tf.initializers.he_normal())
        return recon

In [ ]:
def train_input_fn(features, batch_size=BATCH_SIZE):
    with graph.as_default():
        dataset = tf.data.Dataset.from_tensor_slices(features)
        batch_dataset = dataset.shuffle(features[0].shape[0]).repeat().batch(batch_size)
        return batch_dataset.make_one_shot_iterator().get_next()

<img src="./img/vae_recloss.png" alt="vae_recloss" width="400" align="top"/>
<img src="./img/vae_regloss.png" alt="vae_regloss" width="450" align="top"/>

In [ ]:
def vae_loss(features, outputs, latent_mean, latent_log_var):
    rec_loss = tf.reduce_sum(
        tf.keras.backend.binary_crossentropy(features, outputs), 1)
    reg_loss = -0.5 * tf.reduce_sum(
        1 + latent_log_var 
        - tf.square(latent_mean) 
        - tf.exp(latent_log_var), 1)
    return tf.reduce_mean(rec_loss + reg_loss)

In [ ]:
def create_sprite_image(images):
    img_h = images.shape[1]
    img_w = images.shape[2]
    n_plots = int(np.ceil(np.sqrt(images.shape[0])))
    sprite_image = np.ones((img_h * n_plots ,img_w * n_plots ))    
    for i in range(n_plots):
        for j in range(n_plots):
            this_filter = i * n_plots + j
            if this_filter < images.shape[0]:
                this_img = images[this_filter]
                sprite_image[i * img_h:(i + 1) * img_h,
                  j * img_w:(j + 1) * img_w] = this_img
    plt.imsave(CKPT_DIR + '/sprites.png', sprite_image, cmap='gray_r')

In [ ]:
def create_metadata(labels):
    with open(CKPT_DIR + '/metadata.tsv','w') as f:
        f.write("Index\tLabel\n")
        for i in range(labels.shape[0]):
            f.write("%d\t%d\n" % (i, labels[i]))

In [ ]:
def train(features):
    if not os.path.exists(os.path.dirname(CKPT_DIR)):
        os.makedirs(os.path.dirname(CKPT_DIR))

    with graph.as_default():
        features = train_input_fn(features)
        latent_mean, latent_log_var = encoder_model(features[0])
        sample = sampler_model(latent_mean, latent_log_var)
        outputs = decoder_model(sample)
        loss = vae_loss(features[0], outputs, latent_mean, latent_log_var)
        optimizer = tf.train.AdamOptimizer(LEARNING_RATE).minimize(loss)
        
        
        emb_latent = tf.get_variable('latent', shape=[BATCH_SIZE, LATENT_DIM], dtype=tf.float32)
        emb_latent_assign = tf.assign(emb_latent, sample)
        config = projector.ProjectorConfig()
        embedding = config.embeddings.add()
        embedding.tensor_name = emb_latent.name
        embedding.metadata_path = 'metadata.tsv'
        embedding.sprite.image_path = 'sprites.png'
        embedding.sprite.single_image_dim.extend([28,28])
        org_image = tf.reshape(features[0], [-1, 28, 28, 1])
        rec_image = tf.reshape(outputs, [-1, 28, 28, 1])
        tf.summary.scalar('loss', loss)
        tf.summary.image('org_image', org_image, max_outputs=1)            
        tf.summary.image('rec_image', rec_image, max_outputs=1)
        merged = tf.summary.merge_all()
        saver = tf.train.Saver()
        emb_saver = tf.train.Saver([emb_latent])
        with tf.Session() as sess:
            summary_writer = tf.summary.FileWriter(CKPT_DIR, sess.graph)
            sess.run(tf.global_variables_initializer())

            for step in range(TRAINING_STEPS):
                train_step = step + 1
                train_sample = train_step * BATCH_SIZE
                train_epoch = train_sample / 60000
                _, emb_img, emb_label, _ = sess.run([optimizer, rec_image, features[1], emb_latent_assign])
                if (train_step % (TRAINING_STEPS/10) == 0):
                    summary = sess.run(merged)
                    summary_writer.add_summary(summary, step)
                if (train_step == TRAINING_STEPS):
                    emb_saver.save(sess, CKPT_DIR + '/emb.ckpt')
                    create_sprite_image(emb_img.reshape([-1,28,28]))
                    create_metadata(emb_label)
                    projector.visualize_embeddings(summary_writer, config)
                    saver.save(sess, CKPT_DIR + '/vae.ckpt')
                    
                progress_bar(
                    train_step, 
                    TRAINING_STEPS, 
                    prefix='>>> Training', 
                    suffix='steps: %i/%i, samples: %i/%i, epochs: %i/%i' % (
                        train_step, 
                        TRAINING_STEPS,
                        train_sample, 
                        TRAINING_SAMPLES,
                        train_epoch,
                        TRAINING_EPOCHS))
                
            print('>>> Training Done')

In [ ]:
(x_train, y_train), (x_test, _) = tf.keras.datasets.mnist.load_data()
x_train = x_train / 255.
x_test = x_test / 255.
x_train = x_train.reshape([-1, IMAGE_DIM]).astype(np.float32)
x_test = x_test.reshape([-1, IMAGE_DIM]).astype(np.float32)

In [ ]:
train((x_train, y_train))

In [ ]:
def random_25_image_plot(seed=None):
    with graph.as_default():
        np.random.seed(seed)
        random_noise = np.random.normal(size=[25, LATENT_DIM]).astype(np.float32)
        random_noise_input = train_input_fn(random_noise, batch_size=25)
        random_gen = decoder_model(random_noise_input)

        fig = plt.figure(figsize=(10, 10))
        gs = gridspec.GridSpec(5, 5)
        gs.update(wspace=0.05)

        saver = tf.train.Saver()
        with tf.Session() as sess:
            saver.restore(sess, tf.train.latest_checkpoint(CKPT_DIR))
            random_image = sess.run(random_gen)
            random_image = random_image.reshape([-1, 28, 28])
            for i in range(25):
                plt.subplot(gs[i])
                plt.axis('off')
                plt.imshow(random_image[i], cmap = 'gray')

In [ ]:
random_25_image_plot()